# Coin Classification using Convolutional Network

In [2]:
import os
import cv2
import imutils
import numpy as np
from random import shuffle
import h5py
from pathlib import Path
import tensorflow as tf


/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Dataset creation.

Functions flip, rotate are for expanding coins dataset with modifying coins image by flipping and rotating. Folders Flipped, Rotated should be __existing in the same file system with this project__.

Functions names and short explanation: 
- Images : 2629 coins images. 180 of 2629 are coins from Roman times.
- Flipped: 10516 with flipped coins images. - For each coin image add 4 images of flipped coin versions.
- Rotated: 21032 with rotated coins images. - For each coin image add 8 images of rotated coin versions.

Dataset consist of 34177 images number representation which is fallowing this order: 2629(Base coin images) + 10516(Flipped coins images) + 21032(Rotated coins images).

Images which starts with substring __'class'__ are classified as a Roman coins.

All images are resized to 28x28 pixels.


In [18]:
files = [file for file in os.listdir("Images") if file.endswith(".png") or file.endswith(".jpg")]
for i, file in enumerate(files):
    print("Flipping file {}/{}".format(i,len(files)))
    print(file[:-4])

Flipping file 0/2629
gt_l
Flipping file 1/2629
ds02a_r
Flipping file 2/2629
5s50_r
Flipping file 3/2629
class235_image2
Flipping file 4/2629
6d81_2_r
Flipping file 5/2629
1pdfr_l
Flipping file 6/2629
3d (4)_r
Flipping file 7/2629
c2hc_2_r
Flipping file 8/2629
20s (2)_l
Flipping file 9/2629
class69_image1
Flipping file 10/2629
class349_image1
Flipping file 11/2629
4d (11)_l
Flipping file 12/2629
108_r
Flipping file 13/2629
hg29e_l
Flipping file 14/2629
fa75_2_l
Flipping file 15/2629
11009608_r
Flipping file 16/2629
11009609_l
Flipping file 17/2629
1sgab2_2_r
Flipping file 18/2629
3d52_2_l
Flipping file 19/2629
e13d_2_r
Flipping file 20/2629
john5bl_2_l
Flipping file 21/2629
syria (1)
Flipping file 22/2629
5pd15lrm_l
Flipping file 23/2629
1d53_2_l
Flipping file 24/2629
2g40_l
Flipping file 25/2629
1d26 (1)_2_l
Flipping file 26/2629
p1106_2_l
Flipping file 27/2629
5pd09hp_l
Flipping file 28/2629
1010359_2_r
Flipping file 29/2629
scdcr_r
Flipping file 30/2629
class234_image1
Flipping file 

Flipping file 1183/2629
376_l
Flipping file 1184/2629
2g (4)_1_r
Flipping file 1185/2629
hc47p_2_r
Flipping file 1186/2629
1010120_l
Flipping file 1187/2629
50_08n_r
Flipping file 1188/2629
2g11_r
Flipping file 1189/2629
class68_image2
Flipping file 1190/2629
25613_2_r
Flipping file 1191/2629
5_96_l
Flipping file 1192/2629
104_l
Flipping file 1193/2629
6d46_2_r
Flipping file 1194/2629
2g (3)_l
Flipping file 1195/2629
6d23_2_l
Flipping file 1196/2629
class202_image1
Flipping file 1197/2629
2s31_2_l
Flipping file 1198/2629
ha71_2_r
Flipping file 1199/2629
5s (12)_r
Flipping file 1200/2629
tg_l
Flipping file 1201/2629
494_l
Flipping file 1202/2629
1g (5)_2_l
Flipping file 1203/2629
uni43_r
Flipping file 1204/2629
50_13ci
Flipping file 1205/2629
hc17_2_r
Flipping file 1206/2629
4d3_2_l
Flipping file 1207/2629
fa (7)_1_r
Flipping file 1208/2629
1pd11c
Flipping file 1209/2629
50swim
Flipping file 1210/2629
hg96e_r
Flipping file 1211/2629
6d46l_2_l
Flipping file 1212/2629
sweden1 (1)
Flipping

In [3]:
imageSize = 28
roman_index = []

def flip():
    global roman_index
    files = [file for file in os.listdir("Images") if file.endswith(".png") or file.endswith(".jpg")]
    
    for i, file in enumerate(files):
        print("Flipping file {}/{}".format(i,len(files)))
        
        if(file.startswith('class')):
            roman_index += [i]

        img = cv2.imread("Images/"+file,-1)
        rflip = cv2.flip(img,1)
        vflip = cv2.flip(img,0)
        rvflip = cv2.flip(rflip,0)

        cv2.imwrite("Flipped/"+file[:-4]+"_.png",img)
        cv2.imwrite("Flipped/"+file[:-4]+"_r.png",rflip)
        cv2.imwrite("Flipped/"+file[:-4]+"_v.png",vflip)
        cv2.imwrite("Flipped/"+file[:-4]+"_rv.png",rvflip)
# Generates 2 rotations of the images
def rotate():
    global roman_index
    files = [file for file in os.listdir("Flipped") if file.endswith(".png") or file.endswith(".jpg")]

    # 2 rotations * 4 flips = 8 possibilites
    for index, file in enumerate(files):
        print("Rotating file {}/{}".format(index,len(files)))  
        if(file.startswith('class')):
            roman_index += [index]
        img = cv2.imread("Flipped/"+file,-1)
        cv2.imwrite("Rotated/"+file[:-4]+"_0.png",img)
        img = np.rot90(img)
        cv2.imwrite("Rotated/"+file[:-4]+"_90.png",img)

# Save h5 dataset
def save():
    global roman_index
    #Load, normalize and reshape
    files = os.listdir("Rotated/")
    files = [file for file in files if file.endswith(".png")]

    shuffle(files)
    X = []
    for i, file in enumerate(files):
        print("Adding file {}/{}".format(i,len(files)))
        
        if(file.startswith('class')):
            roman_index += [i]
            
        img = cv2.imread("Rotated/"+file, -1)

        # Convert grayscale to RGB
        if len(img.shape) == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

        # Resize to desired size and normalize
        img = cv2.resize(img,(imageSize,imageSize))
        img = img.astype(float)/255.
        X.append(img)

    # Convert to numpy array
    X = np.array(X)

    # 10% for testing
    testProportion = 0.1
    trainNb = int(X.shape[0]*(1-testProportion))
    testNb = X.shape[0] - trainNb

    print("Splitting dataset in X_train({}) and X_test({})...".format(trainNb,testNb))
    # Split train test
    X_train = X[:trainNb]
    X_test = X[-testNb:]

    #Save at hdf5 format
    print("Saving dataset in hdf5 format... - this may take a while")
    h5f = h5py.File("coins.h5", 'w')
    h5f.create_dataset('X_train', data=X_train)
    h5f.create_dataset('X_test', data=X_test)
    h5f.create_dataset('roman_coins', data=roman_index)
    h5f.close()

Dataset is saved as h5 extension. 
Full generated dataset name: "coins.h5".

If coins.h5 file not exist in the current project file system, functions for dataset generation(flip, rotate and save) are executed. Otherwise all required components are extracted from generated dataset.

### Required components:
- X_train : Coins images representation in RGB(Red, Green, Blue) number notation for each image pixel - used for training Convolution Network.
- X_test : Coins images representation in RGB(Red, Green, Blue) number notation for each image pixel - used for testing Convolution Network performance.
- roman_coins: A list of indexes of Roman coins - used for classification of Roman and non Roman coins.

Training and testing data was splitted into two parts: Training data consist of 18928 units of coins representation data and testing data has 2104 units of coins representation data.


In [4]:
file = Path("coins.h5")
if not file.is_file():
    print("This script creates 21208k images from the base 2652")
    print ("Make sure the folders Rotated and Flipped exist")
    flip()
    rotate()
    save()
X_train = h5py.File('coins.h5','r')['X_train']
X_test = h5py.File('coins.h5','r')['X_test']
coins_index = h5py.File('coins.h5','r')['roman_coins']

## Data preprocessing.

Since features training and testing data is already available from h5 dataset, it is necesarry convert data to Python friendly data structure - numpy array list. 
<br>
For classifcation part, it is needed to extract values from h5 dataset with indexes of Roman coins and preprocess this data for all available coins. 

### Classification
A new list witch contains an index of Roman coins is created to extract and save values from h5 dataset to a python based structure. When this step is done, a labels variables y_train and y_test is created which stores each picture classification value:
- 0 : Representation of non Roman coin.
- 1 : Representation of Roman coin.

In [5]:
roman_coins = []
for i in coins_index:
    roman_coins += [i]

y_train = []
y_test = []
test_limit = len(X_train)
for i, data in enumerate(X_train):
    if i in roman_coins:
        y_train += [1]
    else:
        y_train += [0]

for i, data in enumerate(X_test):
    if i + test_limit in roman_coins:
        y_test += [1]
    else:
        y_test += [0]

y_train = np.array(y_train)
y_test = np.array(y_test)

### Features extraction from h5 dataset
All images are traversed and stored to the Python data structure for Convolution Network processing.

In [6]:
x_train = np.ndarray(shape=X_train.shape, dtype=float)
x_test = np.ndarray(shape=X_test.shape, dtype=float)
for i, data in enumerate(X_train):
    x_train[i] = data
for i, data in enumerate(X_test):
    x_test[i] = data

## Convolution Network
Parameters short explanation for Convolution Network:
- __Height and Width__ - Image height and weight. Our images are made of 28*28 pixels, so as a height and width numbers are the same.
- __Channels__ - Represents image color set. In our project images consist of combination of three colors: red, green, blue(RGB). In this case channels value is 3.
- __n_inputs__ - Number of inputs which are calculated by multiplying height, width and channel number. In our project inputs value is 2352(28*28*3).
- __ntrain__ - Number of all instances of training set (18928).
- __ntest__ - Number of all instances of testing set (2104).
- __conv1_fmaps, conv2_fmaps__ - Feature map for the first(conv1_fmaps) and second(conv2_fmaps) layers. The feature map is the output of one filter applied to the previous layer. A given filter is drawn across the entire previous layer, moved one pixel at a time. Each position results in an activation of the neuron and the output is collected in the feature map.
- __conv1_ksize, conv2_ksize__ - Kernel size for first and second layers. Kernels are used for feature detection.
- __conv1_stride, conv2_stride__ - The amount of shifts.
- __conv1_pad, conv2_pad__ - Padding parameters. Parameter value is equal 'Same' which is Trying to pad evenly left and right, but if the amount of columns to be added is odd, it will add the extra column to the right
- __pool3_fmaps__ - Pooling feature map.
- __n_fc1__ - Fully connected layer. Fully connected layer looks at what high level features most strongly correlate to a particular class and has particular weights so that when you compute the products between the weights and the previous layer, you get the correct probabilities for the different classes.
- __n_outputs__ - Number of classifiers. Outputs value equals to 2, because there are two possible prediction outputs: 0(Non Roman coin) and 1(Roman coin). 

Convolution Network has 2 convolution layers: __conv1 and conv2__. These layers create a convolution kernels that are convolved with the layer input to produce a tensor of outputs.
<br>




In [7]:
# We define the characteristics of the input image 
height = 28
width = 28
channels = 3
n_inputs = height * width * 3
ntrain = len(X_train)
ntest  = len(X_test)

# We define the parameters of the layers according to 
# description previously presented

conv1_fmaps = 16       #32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 24     #64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 2


with tf.name_scope("inputs"):
    # Variable X is passed as a vector
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    # It is reshaped to the tensor according to image size an channels
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    # Class of each MNIST image
    y = tf.placeholder(tf.int32, shape=[None], name="y")

# The first layer is defined. Notice that the tensorflow function used
# is tf.layers.conv2d(). Also the parameters are those previously defined.


conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")

# The second layer is defined. Notice that the input of this layer is the output
# of the previous layer. You can check that conv1 has size 28x28 and conv2 has
# size 14x14 (Try to find out why)

conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

# The maxpool layer is defined. Notice that the  tf.nn.max_pool() is used to define the layer.
# Also, maxpool is applied to each of the conv2_fmaps filters, and since the inputs have size
# 14x14, Stride=2 and Padding=Valid, after applying maxpool we have pool3_fmaps filters
# of size (7x7). That is the reason while the output is reshaped (flattened) to (pool3_fmaps * 7 * 7)

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 7 * 7])

# This is the full layer. From previous classes we already know function tf.layers.dense()
# used to define full layers. 
# The number of input and output units is the same (pool3_fmaps * 7 * 7)
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

# This is the output layer where the network produces a classification for each class
# The classification is used using the function softmax that we have studied in the previous lab
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

# The loss function and optimizers are defined as in previous labs.    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

# We define two functions to evaluate the quality of the network as classifier
# Correct computes, for a batch of observations, how many were correctly classified.
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# We define a saver 
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [10]:
print(ntrain)

18928


In [16]:
n_epochs = 10
IMAGE_SIZE = 28
NUM_CHANNELS = 3
BATCH_SIZE = 100
test_size = ntest

config = tf.ConfigProto(device_count = {'GPU': 0})
with  tf.Session(config=config) as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(ntrain // BATCH_SIZE):
            randidx = np.random.randint(ntrain, size=BATCH_SIZE)
            X_batch = x_train[randidx, :]
            y_batch = y_train[randidx]  
            X_batch = np.reshape(X_batch, (-1, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})


        correct_pred = 0
        for iteration in range(ntest // BATCH_SIZE):
            randidx = np.random.randint(ntest, size=BATCH_SIZE)
            X_batch = x_test[randidx, :]
            y_batch = y_test[randidx]  
            X_batch = np.reshape(X_batch, (-1, n_inputs))
            correct_pred += np.sum(correct.eval(feed_dict={X: X_batch, y: y_batch}))
#             print(correct_pred)
#             print((correct_pred / float(test_size)))
#         acc_test = (correct_pred / float(test_size))
        print(correct_pred / float(test_size))
        acc_test = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

        #save_path = saver.save(sess, "./my_mnist_model")

0.951996197719
0 Train accuracy: 0.92 Test accuracy: 0.95
0.958650190114
1 Train accuracy: 0.9 Test accuracy: 0.97
0.973384030418
2 Train accuracy: 0.95 Test accuracy: 0.96
0.979562737643
3 Train accuracy: 0.93 Test accuracy: 0.99
0.985266159696
4 Train accuracy: 0.98 Test accuracy: 0.99
0.980988593156
5 Train accuracy: 0.92 Test accuracy: 0.98
0.983840304183
6 Train accuracy: 0.94 Test accuracy: 0.99
0.992395437262
7 Train accuracy: 0.94 Test accuracy: 1.0
0.992395437262
8 Train accuracy: 0.95 Test accuracy: 1.0
0.990019011407
9 Train accuracy: 0.96 Test accuracy: 1.0
